# Regras de Associação

***


In [22]:
import warnings
warnings.filterwarnings('ignore')

## Análise do conjunto de dados

Regras de associação são algoritmos que extraem conjuntos de itens frequentes em datasets que cada instância é um conjunto de itens. Vamos visualizar o que isso significa.

Vamos observar o dataset de compras de um supermercado. Segue o link para mais informações [dataset](https://archive.ics.uci.edu/dataset/352/online+retail)

In [23]:
# Carregando o dataset

import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/Francimaria/monitoria-ml/main/online_retail.csv")


Vamos visualizar uma pequena parte desse dataset.

In [10]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


Aqui temos diversas informações sobre compras em um supermercado:

- (_InvoiceNo_) o número da fatura, identificador de uma compra
- (_StockCode_) o código de certo produto no estoque
- (_Description_) a descrição do produto
- (_Quantity_) a quantidade de produtos que foi comprada
- (_InvoiceDate_) o dia da compra
- (_UnitPrice_) o preço por unidade
- (_CustomerID_) o id do consumidor
- (_Country_) o país de venda

In [24]:
# vamos olhar todos os países existentes
print(df["Country"].unique())
# print(df["Country"].value_counts())

# dos países, vamos escolher apenas as vendas feitas no Reino Unido, apenas pelo fato de existirem mais vendas no dataset
country = "United Kingdom"
sales = df[df["Country"] == country]

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


In [25]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495478 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    495478 non-null  object 
 1   StockCode    495478 non-null  object 
 2   Description  494024 non-null  object 
 3   Quantity     495478 non-null  int64  
 4   InvoiceDate  495478 non-null  object 
 5   UnitPrice    495478 non-null  float64
 6   CustomerID   361878 non-null  float64
 7   Country      495478 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 34.0+ MB


In [43]:
"C - indica cancelamento"
sales[sales["InvoiceNo"].str.contains("C")].head(10)

# sales[sales["Description"].isna()]



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


In [44]:
# antes de utilizar nosso dataset, precisamos fazer alguns pre-processamentos:
# remover todas as instâncias que não possuem description
filtered_sales = sales.dropna(axis=0, subset=["Description"]) 
print(filtered_sales.shape)
# transformar em strings
filtered_sales["InvoiceNo"] = filtered_sales["InvoiceNo"].astype("str")
# remover instancias que contenham "C" no _InvoiceNo_, representando instancias que não possuem essa feature
filtered_sales = filtered_sales[~filtered_sales["InvoiceNo"].str.contains("C")]

# remover espaços desnecessários no começo e fim de cada _Description_
filtered_sales["Description"] = filtered_sales["Description"].str.strip()
filtered_sales["Description"] = filtered_sales["Description"].astype("str")


(494024, 8)


Como o dataset a ser avaliado deverá consistir em conjuntos de itens, vamos transformar cada compra em um conjunto de itens. Note que um conjunto não possui informação sobre quantidade de elementos repetidos. Portanto, a coluna com a contagem de cada item deve ser removida, indicando apenas a presença daquele item em uma compra.

In [45]:
sales_set = filtered_sales.groupby(['InvoiceNo', 'Description'])["Quantity"].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
sales_set.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
sales_set[["POSTAGE", "DOTCOM POSTAGE"]]

Description,POSTAGE,DOTCOM POSTAGE
InvoiceNo,,
536365,0.0,0.0
536366,0.0,0.0
536367,0.0,0.0
536368,0.0,0.0
536369,0.0,0.0
...,...,...
581585,0.0,0.0
581586,0.0,0.0
A563185,0.0,0.0


In [47]:
# existem algumas vendas que tiveram algum tipo de problema, vamos remove-las a partir no index 4057

#vamos reduzir o número de colunas para 1500
sales_set = sales_set.iloc[:, :1500]
print(sales_set.shape)

# também estão descritos o tipo de postagem, se pela internet ou não, vamos remove-los
sales_set = sales_set.drop("POSTAGE", axis=1, errors="ignore")
sales_set = sales_set.drop("DOTCOM POSTAGE", axis=1, errors="ignore")

# nosso dataset final
sales_set.head()

(18668, 1500)


Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,GLASS CHALICE GREEN SMALL,GLASS CLOCHE LARGE,GLASS CLOCHE SMALL,GLASS HEART T-LIGHT HOLDER,GLASS JAR DAISY FRESH COTTON WOOL,GLASS JAR DIGESTIVE BISCUITS,GLASS JAR ENGLISH CONFECTIONERY,GLASS JAR KINGS CHOICE,GLASS JAR MARMALADE,GLASS JAR PEACOCK BATH SALTS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# agora, vamos transformar o dataset de uma contagem de itens, para apenas conjuntos
# faremos isso transformando toda contagem para uma presença ou não do item

count_to_set = lambda x: x > 0 # aqui se uma venda possui pelo menos 1 item, afime True, caso contrário, False
sales_set = sales_set.applymap(count_to_set)

print(sales_set.shape)
sales_set.head()

(18668, 1499)


Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,GLASS CHALICE GREEN SMALL,GLASS CLOCHE LARGE,GLASS CLOCHE SMALL,GLASS HEART T-LIGHT HOLDER,GLASS JAR DAISY FRESH COTTON WOOL,GLASS JAR DIGESTIVE BISCUITS,GLASS JAR ENGLISH CONFECTIONERY,GLASS JAR KINGS CHOICE,GLASS JAR MARMALADE,GLASS JAR PEACOCK BATH SALTS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Após esse paço de pre-processamento, conseguimos extrair quais itens são comprados em conjunto com maior frequência para todas as vendas.

## Apriori

Inicialmente, vamos utilizar o algoritmo a priori. Nele precisamos indicar qual a repetição mínima necessária de repetição que buscamos. Por exemplo, se quisermos apenas as repetições que aconteçam pelo menos $n\%$ das vezes.

O algoritmo funciona por criar todas as combinações possíveis de conjuntos e então checar suas frequências.

In [32]:
from mlxtend.frequent_patterns import apriori

# use_colnames retorna o itemset como nomes ao invés de indices das colunas
frequency_set = apriori(sales_set, min_support=0.01, use_colnames=True)
print(frequency_set)

#Unable to allocate 9.89 GiB for an array with shape (263901, 2, 20122) and data type bool

      support                                           itemsets
0    0.014624                           (10 COLOUR SPACEBOY PEN)
1    0.012910                  (12 MESSAGE CARDS WITH ENVELOPES)
2    0.017517                    (12 PENCIL SMALL TUBE WOODLAND)
3    0.018159              (12 PENCILS SMALL TUBE RED RETROSPOT)
4    0.018052                      (12 PENCILS SMALL TUBE SKULL)
..        ...                                                ...
283  0.012374  (FELTCRAFT PRINCESS CHARLOTTE DOLL, FELTCRAFT ...
284  0.028980  (GARDENERS KNEELING PAD CUP OF TEA, GARDENERS ...
285  0.010660  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...
286  0.012696  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...
287  0.013713  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...

[288 rows x 2 columns]


In [33]:
# perceba que temos muitas conjuntos com apenas um elemento
# vamos filtrar esses conjuntos que tem apenas um elemento

def filter_set_lenght(input_set, lenght=2):
    input_set['set_lenght'] = input_set['itemsets'].apply(lambda x: len(x))
    new_set = input_set[input_set['set_lenght'] >= lenght]
    new_set.reset_index(inplace=True, drop=True)
    return new_set
    
combo_set = filter_set_lenght(frequency_set)
print(combo_set)

     support                                           itemsets  set_lenght
0   0.013338  (60 TEATIME FAIRY CAKE CASES, 72 SWEETHEART FA...           2
1   0.013338  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
2   0.013606  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
3   0.016392  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
4   0.012963  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
5   0.018749  (ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...           2
6   0.030159  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
7   0.013285  (ALARM CLOCK BAKELIKE IVORY, ALARM CLOCK BAKEL...           2
8   0.018320  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
9   0.014195  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
10  0.021052  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...           2
11  0.015106  (BAKING SET 9 PIECE RETROSPOT, BAKING SET SPAC...           2
12  0.010392

In [49]:
# escolhendo a combinação de itens com maior repetição
def get_highest_support(input_set):
    instance = input_set.iloc[input_set["support"].idxmax()]
    return instance

# e criando uma função auxiliar para mostrar as estatísticas do nosso combo a partir do nosso conjunto de vendas
def print_combo_stats(combo, sales):
    print("itens: %s"%(str(tuple(combo["itemsets"]))))
    print("frequencia %.2f%%" %(float(combo["support"])*100))
    print("vendas totais do combo: %d"%(float(combo["support"]) * sales.shape[0]))

combo = get_highest_support(combo_set)
print_combo_stats(combo, sales_set)

itens: ('ALARM CLOCK BAKELIKE RED', 'ALARM CLOCK BAKELIKE GREEN')
frequencia 3.02%
vendas totais do combo: 563


## FP-Growth

Ao contrário do algoritmo a priori, FP-Growth não precisa criar todas os conjuntos de combinações. Para datasets em que a quantidade de combinações é muito grande, ele possui uma vantagem sobre seu tempo de execução.

In [50]:
# caso o import fpgrowth falhe, descomente a linha abaixo
# !pip install mlxtend -U

from mlxtend.frequent_patterns.fpgrowth import fpgrowth

# use_colnames retorna o itemset como nomes ao invés de indices das colunas
frequency_set = fpgrowth(sales_set, min_support=0.01, use_colnames=True)
print(frequency_set)

      support                                           itemsets
0    0.014356                   (CREAM CUPID HEARTS COAT HANGER)
1    0.073441                    (ASSORTED COLOUR BIRD ORNAMENT)
2    0.030105                              (DOORMAT NEW ENGLAND)
3    0.022123                (FELTCRAFT PRINCESS CHARLOTTE DOLL)
4    0.012696                   (BOX OF VINTAGE ALPHABET BLOCKS)
..        ...                                                ...
283  0.012213  (CHARLOTTE BAG APPLES DESIGN, CHARLOTTE BAG PI...
284  0.011571  (CHARLOTTE BAG APPLES DESIGN, CHARLOTTE BAG VI...
285  0.012428  (CHARLOTTE BAG SUKI DESIGN, CHARLOTTE BAG VINT...
286  0.010446  (CHARLOTTE BAG VINTAGE ALPHABET, CHARLOTTE BAG...
287  0.028980  (GARDENERS KNEELING PAD CUP OF TEA, GARDENERS ...

[288 rows x 2 columns]


In [51]:
combo_set = filter_set_lenght(frequency_set)
combo = get_highest_support(combo_set)
print_combo_stats(combo, sales_set)

itens: ('ALARM CLOCK BAKELIKE RED', 'ALARM CLOCK BAKELIKE GREEN')
frequencia 3.02%
vendas totais do combo: 563


Temos o mesmo resultado, mas vamos avaliar o tempo de execução entre cada algoritmo...

In [52]:
from time import time
t0 = time()
frequency_set = apriori(sales_set, min_support=0.01, use_colnames=True)
t1 = time()
frequency_set = fpgrowth(sales_set, min_support=0.01, use_colnames=True)
t2 = time()

print("APriori(t_delta): %f" %(t1-t0))
print("FP-Growth(t_delta): %f" %(t2-t1))

APriori(t_delta): 1.418198
FP-Growth(t_delta): 0.956812
